In [1]:
!git clone https://github.com/n00bmaster68/ApplyingSODinAutoLabelling


Cloning into 'ApplyingSODinAutoLabelling'...
remote: Enumerating objects: 2084, done.
remote: Counting objects: 100% (2084/2084), done.
remote: Compressing objects: 100% (2064/2064), done.
remote: Total 2084 (delta 15), reused 2073 (delta 8), pack-reused 0
Receiving objects: 100% (2084/2084), 74.94 MiB | 31.35 MiB/s, done.
Resolving deltas: 100% (15/15), done.


In [2]:
#check whether device has GPU
import tensorflow as tf
tf.test.gpu_device_name()
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-a3e11364-69ae-12f5-905e-a86de5192f90)


In [3]:
import torch
import torchvision
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import torch.optim as optim
import torchvision.transforms as standard_transforms

import numpy as np
import glob

from ApplyingSODinAutoLabelling.data_loader import Rescale
from ApplyingSODinAutoLabelling.data_loader import RescaleT
from ApplyingSODinAutoLabelling.data_loader import RandomCrop
from ApplyingSODinAutoLabelling.data_loader import CenterCrop
from ApplyingSODinAutoLabelling.data_loader import ToTensor
from ApplyingSODinAutoLabelling.data_loader import ToTensorLab
from ApplyingSODinAutoLabelling.data_loader import SalObjDataset

from ApplyingSODinAutoLabelling.model import BASNet

import ApplyingSODinAutoLabelling.deletable.pytorch_ssim as pytorch_ssim
import ApplyingSODinAutoLabelling.deletable.pytorch_iou as pytorch_iou

In [4]:
# ------- 1. define loss function --------

bce_loss = nn.BCELoss(size_average=True)
ssim_loss = pytorch_ssim.SSIM(window_size=11,size_average=True)
iou_loss = pytorch_iou.IOU(size_average=True)

def bce_ssim_loss(pred,target):

	bce_out = bce_loss(pred,target)
	ssim_out = 1 - ssim_loss(pred,target)
	iou_out = iou_loss(pred,target)

	loss = bce_out + ssim_out + iou_out

	return loss

def muti_bce_loss_fusion(d0, d1, d2, d3, d4, d5, d6, d7, labels_v):

	loss0 = bce_ssim_loss(d0,labels_v)
	loss1 = bce_ssim_loss(d1,labels_v)
	loss2 = bce_ssim_loss(d2,labels_v)
	loss3 = bce_ssim_loss(d3,labels_v)
	loss4 = bce_ssim_loss(d4,labels_v)
	loss5 = bce_ssim_loss(d5,labels_v)
	loss6 = bce_ssim_loss(d6,labels_v)
	loss7 = bce_ssim_loss(d7,labels_v)
	#ssim0 = 1 - ssim_loss(d0,labels_v)

	# iou0 = iou_loss(d0,labels_v)
	#loss = torch.pow(torch.mean(torch.abs(labels_v-d0)),2)*(5.0*loss0 + loss1 + loss2 + loss3 + loss4 + loss5) #+ 5.0*lossa
	loss = loss0 + loss1 + loss2 + loss3 + loss4 + loss5 + loss6 + loss7#+ 5.0*lossa
	print("l0: %3f, l1: %3f, l2: %3f, l3: %3f, l4: %3f, l5: %3f, l6: %3f\n"%(loss0.data,loss1.data,loss2.data,loss3.data,loss4.data,loss5.data,loss6.data))
	# print("BCE: l1:%3f, l2:%3f, l3:%3f, l4:%3f, l5:%3f, la:%3f, all:%3f\n"%(loss1.data[0],loss2.data[0],loss3.data[0],loss4.data[0],loss5.data[0],lossa.data[0],loss.data[0]))

	return loss0, loss

/usr/local/lib/python3.7/dist-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))


In [5]:
# ------- 2. set the directory of training dataset --------

data_dir = 'ApplyingSODinAutoLabelling/train_data/'
tra_image_dir = 'DUTS/DUTS-TR/DUTS-TR/im_aug/'
tra_label_dir = 'DUTS/DUTS-TR/DUTS-TR/gt_aug/'

image_ext = '.jpg'
label_ext = '.png'

model_dir = "ApplyingSODinAutoLabelling/saved_models/basnet_bsi/"


epoch_num = 100000
batch_size_train = 8 
batch_size_val = 1
train_num = 0
val_num = 0

tra_img_name_list = glob.glob(data_dir + tra_image_dir + '*' + image_ext)

tra_lbl_name_list = []
for img_path in tra_img_name_list:
	img_name = img_path.split("/")[-1]

	aaa = img_name.split(".")
	bbb = aaa[0:-1]
	imidx = bbb[0]
	for i in range(1,len(bbb)):
		imidx = imidx + "." + bbb[i]

	tra_lbl_name_list.append(data_dir + tra_label_dir + imidx + label_ext)

print("---")
print("train images: ", len(tra_img_name_list))
print("train labels: ", len(tra_lbl_name_list))
print("---")

train_num = len(tra_img_name_list)

salobj_dataset = SalObjDataset(
    img_name_list=tra_img_name_list,
    lbl_name_list=tra_lbl_name_list,
    transform=transforms.Compose([
        RescaleT(256),
        RandomCrop(224),
        ToTensorLab(flag=0)]))
salobj_dataloader = DataLoader(salobj_dataset, batch_size=batch_size_train, shuffle=True, num_workers=1)


---
train images:  1000
train labels:  1000
---


In [6]:
# ------- 3. define model --------
# define the net
net = BASNet(3, 1)
if torch.cuda.is_available():
    net.cuda()

Downloading: "https://download.pytorch.org/models/resnet34-333f7ec4.pth" to /root/.cache/torch/hub/checkpoints/resnet34-333f7ec4.pth


In [7]:
# ------- 4. define optimizer --------
print("---define optimizer...")
optimizer = optim.Adam(net.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0)


---define optimizer...


In [ ]:
# ------- 5. training process --------
print("---start training...")
ite_num = 0
running_loss = 0.0
running_tar_loss = 0.0
ite_num4val = 0

for epoch in range(0, epoch_num):
    net.train()

    for i, data in enumerate(salobj_dataloader):
        ite_num = ite_num + 1
        ite_num4val = ite_num4val + 1

        inputs, labels = data['image'], data['label']

        inputs = inputs.type(torch.FloatTensor)
        labels = labels.type(torch.FloatTensor)

        # wrap them in Variable
        if torch.cuda.is_available():
            inputs_v, labels_v = Variable(inputs.cuda(), requires_grad=False), Variable(labels.cuda(),
                                                                                        requires_grad=False)
        else:
            inputs_v, labels_v = Variable(inputs, requires_grad=False), Variable(labels, requires_grad=False)

        # y zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        d0, d1, d2, d3, d4, d5, d6, d7 = net(inputs_v)
        loss2, loss = muti_bce_loss_fusion(d0, d1, d2, d3, d4, d5, d6, d7, labels_v)

        loss.backward()
        optimizer.step()

        # # print statistics
        running_loss += loss.data
        running_tar_loss += loss2.data

        # del temporary outputs and loss
        del d0, d1, d2, d3, d4, d5, d6, d7, loss2, loss

        print("[epoch: %3d/%3d, batch: %5d/%5d, ite: %d] train loss: %3f, tar: %3f " % (
        epoch + 1, epoch_num, (i + 1) * batch_size_train, train_num, ite_num, running_loss / ite_num4val, running_tar_loss / ite_num4val))

        if ite_num % 2000 == 0:  # save model every 2000 iterations

            torch.save(net.state_dict(), model_dir + "basnet_bsi_itr_%d_train_%3f_tar_%3f.pth" % (ite_num, running_loss / ite_num4val, running_tar_loss / ite_num4val))
            running_loss = 0.0
            running_tar_loss = 0.0
            net.train()  # resume train
            ite_num4val = 0

print('-------------Congratulations! Training Done!!!-------------')


---start training...


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3458: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode)
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1709: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Streaming output truncated to the last 5000 lines.
l0: 0.904926, l1: 0.908631, l2: 0.924341, l3: 0.926049, l4: 0.960971, l5: 1.067154, l6: 1.297785

[epoch:  28/100000, batch:   320/ 1000, ite: 3415] train loss: 8.283353, tar: 0.942940 
l0: 0.711171, l1: 0.711931, l2: 0.724143, l3: 0.737080, l4: 0.767383, l5: 0.885739, l6: 1.049495

[epoch:  28/100000, batch:   328/ 1000, ite: 3416] train loss: 8.282228, tar: 0.942776 
l0: 0.671728, l1: 0.673873, l2: 0.676353, l3: 0.702407, l4: 0.755822, l5: 0.865236, l6: 1.075955

[epoch:  28/100000, batch:   336/ 1000, ite: 3417] train loss: 8.281055, tar: 0.942585 
l0: 0.769748, l1: 0.769483, l2: 0.779167, l3: 0.834984, l4: 0.876621, l5: 0.967100, l6: 1.158175

[epoch:  28/100000, batch:   344/ 1000, ite: 3418] train loss: 8.280348, tar: 0.942463 
l0: 0.619869, l1: 0.620435, l2: 0.626613, l3: 0.640066, l4: 0.661846, l5: 0.699145, l6: 0.876901

[epoch:  28/100000, batch:   352/ 1000, ite: 3419] train loss: 8.278479, tar: 0.942235 
l0: 0.768266, l1: 0